<h1><center>Introduction to Computational Neuroscience</center></h1>
<h1><center> Practice V: Neurons Networks </center></h1>
<center>Daniel Majoral, Aqeel Labash, Raul Vicente</center>

<center> October 16 2018 <center>

### Important:
Make sure that you saved your ipynb file correctly to avoid loss of information. Please submit this **ipynb** file only (unless you have extra files then zip this file with the extra ones). Everything should be included in this file questions, answers, codes, plots, and comments about your solutions.

My **Pseudonym** is: <font color='green'>[YOUR ANSWER]</font> and it took me approximately: <font color='green'>[YOUR ANSWER]</font> hours to complete the home work.

The data of how long it took you to complete the home work will help us to improve the home works to be balanced.

In this session we will have a brief look on how neurons behave if we make them interact
with each other. For this purpose we use the integrate-and-fire neuron with leak currents and
synaptic currents, which was already part of the homework last week. This time I have written
lots of new code for you to run and explore. You will need to change parameters here and there
and draw conclusions from what you see. As there is no coding demanded, the answers and
analysis are expected to be thorough. (HINT if you are not sure about an effect or tendency,
you are free to run more simulations -with different parameter values- than you are asked to)

## Exercise 1.1: Two neurons.


### Part I: competition (1pt)
    


In the lecture you saw microcircuits (canonical circuits) consisting of three neurons. These triplets gave rise to different behaviours,  depending
on how the neurons were connected to each other. In this task we will model a very similar thing, but with only 2 neurons. The effect of
a third neuron is replaced by injected currents.


**Description of neurons:** We have two identical neurons that both receive the same
amount of constant input (I<sub>injected</sub>), the ion pumps in their membranes also give rise to I<sub>leak</sub>.
Importantly, they also receive input from the other neuron, summarized in the current I<sub>syn</sub>.
Starting from the second TODO, we make our neurons also targets of noise-currents (that can
be both negative and positive) I<sub>noise</sub>. In all, the 4 types of currents sum up to give rise to the
changes in the membrane potential.


As you know already from earlier lectures, the synaptic interactions between neurons can
be of excitatory or inhibitory nature. We will first take a look at the case where the neurons
inhibit each other. That means neuron A has an inhibitory synapse on the dendrites of neuron
B and vice versa.

Every time neuron A reaches threshold, an inhibitory synaptic current will flow into neuron
B. This current will lower the membrane potential and reduce the chance of B firing for a period
of time. As a result, neuron A might reach the threshold again (for a second time) before B
even reaches the threshold for the first time. This adds further inhibition to B and makes it's
firing even less probable. Eventually, even though the two identical neurons both receive the
same amount of excitatory injected current, neuron B might never fire at all! Symmetrically, if
it had been neuron B that fired first, it would be the neuron A that will be forever inhibited.
That's why we call this configuration "competition" - the neurons compete with each other on
who has the right to be active (and "the winner takes it all").

In [ ]:
# Adding libraries 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from math import exp,sqrt

In [ ]:
class Neuron(object):
    """Standard neuron for this practice.
    """
    #Constant parameters
    C = 0.1        # capacity
    R = 1          # resistance
      
    Vr = -75.0 # reset potential in mV
    V0 = -70.0 # resting potential in mV

    tc = -100000 
    th = -55.0    #the neuron threshold
    
    g_leak = 1
   
    def __init__(self,spike_strength,I_const,g_leak,taus):
        
        self.V = self.V0 + np.random.normal(1)*5  # Random initial voltage
        
        self.spike_strength = spike_strength # How incoming synapses influence this neuron
        self.t_last_spike = -100 # Synaptic currents want to know the last output spike time
    
        self.Vm = []     # History with all past membrane potentials
        self.spikes = [] # History with all past spikes 0 not spike 1 spike
        
        self.seps = 0 # Current from neighbour neurons spikes
        
        self.I_const = I_const # Current that simulates global activity from other neurons
        self.g_leak =  g_leak
        self.taus = taus # how fast does the synaptic current decays (half life)
    
        self.RC = self.R*self.C
       
    def checkspike(self,t): #Checks if the neuron has generated an spike and updates parameters  
        s = 0 
        if(self.V >= self.th):   
            s = 1
            self.V = self.Vr 
            self.t_last_spike = t  
        
        self.spikes.append(s)   
          
    def synaptic_input(self,input_spikes,t):
        """Calculates the current coming from other neurons spikes"""
        
        self.seps = 0
        for t_spike in input_spikes:
            
            eps = 0
            
            if (self.t_last_spike <= t_spike) and (t_spike <= t): #the last output was long time ago
                    td = t - t_spike
                    eps = exp(-td/self.RC) -exp(-td/self.taus)
                  
            if (t_spike < self.t_last_spike) and (self.t_last_spike <= t): # there has been an output after our input 
                    td1 = self.t_last_spike - t_spike
                    td2 = t - self.t_last_spike
                    eps = exp(-td1/self.taus)*(exp(-td2/self.RC) - exp(-td2/self.taus)) 
                   
            self.seps += eps; # current from spikes
    
    def update_voltage(self,t,noise,dt):
        "Calculates the currents and updates the voltage of the neuron"
            
        I_syn = self.spike_strength * 1/(1-self.taus/self.RC)*self.seps 
        I_noise = noise*sqrt(dt)*np.random.randn()
        
        I_injected = dt*self.I_const/self.C
        I_leak = self.g_leak*dt*(self.V0 - self.V)/(self.RC)
        
        self.V = self.V + I_leak + I_injected + I_syn + I_noise
        self.Vm.append(self.V)
        
    def display(self):
        """Display Neuron parameters."""
        print("\n Neuron Parameters:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")
   
#Function to draw the neurons.          
def Plot_neurons(neurons,colors=['red','blue'],figsize_p=(12,6),p_xlbl='time [ms]',p_ylbl='Vm(t)  [mV]'):
    plt.figure(figsize=figsize_p)  
    for i in range(len(neurons)):
        plt.plot(neurons[i].Vm,color=colors[i])
        for j,v in enumerate(neurons[i].Vm):
            if v>neurons[i].th:
                plt.scatter(j,neurons[i].Vm[j],facecolors='none',color=colors[i])
    plt.xlabel(p_xlbl);
    plt.ylabel(p_ylbl)
    plt.ylim(-100, -50)    
    plt.show()

n1 = Neuron(spike_strength=-200,I_const=50,g_leak=1,taus=20) 
n1.display()

In [ ]:
def competition(noise,spike_strength = -200.0):

    T = 4     #duration of simulation in sec
    dt= 0.001
    I_const = 50.0; # injected current to each cell
    g_leak = 1
    taus = 20
    number_of_steps = int(T/dt)
    
    n_neurons = 2
    
    #How neurons are connected between them if [0,1] == 1 then conexion from neuron 0 to 1     
    connectivity = np.zeros((n_neurons,n_neurons))
    connectivity[0,1] =1 
    connectivity[1,0] =1
    
    #We create the neurons with the class from the previous cell
    neurons = []
    for n in range(n_neurons): 
        neurons.append(Neuron(spike_strength,I_const,g_leak,taus))
   
    print('Initial voltages:',neurons[0].V,neurons[1].V)
                  
    for i in range(number_of_steps): 
        
        for j,ne in enumerate(neurons): 
            
            ne.checkspike(i) #Checks if the neuron has generated one spike
        
            neighbours = np.where(connectivity[j,:] == 1)  #Neurons from wich the neuron receives inputs 
            
            #Calculate incoming spikes 
            in_spikes=[]
            for ngh in neighbours:
                nh_time_spike = np.concatenate(np.nonzero(neurons[ngh[0]].spikes))  #past timepoints with spikes in the neighbour
                in_spikes.append(nh_time_spike)
            in_spikes = np.concatenate(in_spikes)
            
            ne.synaptic_input(in_spikes,i)
            
            ne.update_voltage(i,noise,dt)
           
    for i,ne in enumerate(neurons):        
        print("Neuron {} spike count {}".format(i,sum(ne.spikes)))
  
    return neurons    

In [ ]:
#Example code use
noise = 20
spike_strength = -200
neurons = competition(noise,spike_strength)
Plot_neurons(neurons)

**Q1 :** Run the function competition with the parameter noise equal to 0.0.
Run it multiple times, so that you would see that depending on the initial membrane potential
of the neurons (Vm is initialized randomly), either the first or the second neuron will become
dominant. Produce a plot of both cases.

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

In the above case the noise current was "turned off" and given the same starting membrane
potentials the simulation would always behave exactly the same. Now let's turn the noise up a
little and see how this influences the system.

**Q2 :** Run the function competition with the parameter noise equal to 10.0.
Run it multiple times. You should see that with this amount of noise, sometimes the neurons
switch places during a simulation - the one that was being inhibited can become the active one.
Add a plot from a simulation where the neurons clearly switched places. Why
can this happen now that we added noise (and why it could not happen without noise)?

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

**Q3** Run competition with the parameter noise equal to 10.0 and parameter spike_strength =
-300 (the strength of inhibition). Run it multiple times. Why does the switch not happen any more?

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

**Q4 :** Run competition with the parameter noise equal to 50.0 and parameter spike_strength =-200.0. Why does the switching becomes more frequent with higher noise? Also notice that
the overall behaviour is more jumpy. Plot the behaviour.

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

## Exercise 1.2: Two neurons.

### Part II: oscillation (0.5pt)


In the second part of this exercise we will look at the emergence of oscillations when neuron A
excites neuron B, but neuron B inhibits neuron A.  
In this task only neuron A receives the injected and noise currents. Neuron B will be activated only by the excitatory synaptic currents coming from its synapses with A (A -> B). As
soon as B has accumulated enough excitation and fires, neuron A will be inhibited due to the
inhibitory synapse between B -> A. This will create a pause in neuron A-s firing even though
the excitatory injected current is still there.

In [ ]:
def oscillation(spike_strength =[-500.0,100.0],taus=[15,2]):
    
    T = 2;  #duration of simulation in sec
    dt=0.001;
    number_of_steps = int(T/dt)
    
    I_const = [80.0,0.0] # injected current to each cell
    noise = [10.0,0.0] #no noise for inhibitory
    g_leak= [0.2,0.1]
    taus = [15,2]
    
    n_neurons = 2
    
    #How neurons are connected between them if [0,1] == 1 then conexion from neuron 0 to 1     
    connectivity = np.zeros((n_neurons,n_neurons))
    connectivity[0,1] =1 
    connectivity[1,0] =1
    
    #We create the neurons with the class from the previous cell
    neurons = []
    for i in range(n_neurons): 
        neurons.append(Neuron(spike_strength[i],I_const[i],g_leak[i],taus[i]))
   
    print('Initial voltages:',neurons[0].V,neurons[1].V)
                  
    for i in range(number_of_steps): 
        
        for j,ne in enumerate(neurons): 
            
            ne.checkspike(i) #Checks if the neuron has generated one spike
        
            neighbours = np.where(connectivity[j,:] == 1)  #Neurons from wich the neuron receives inputs 
            
            #Calculate incoming spikes 
            in_spikes=[]
            for ngh in neighbours:
                nh_time_spike = np.concatenate(np.nonzero(neurons[ngh[0]].spikes))  #past timepoints with spikes in the neighbour
                in_spikes.append(nh_time_spike)
            in_spikes = np.concatenate(in_spikes)
            
            ne.synaptic_input(in_spikes,i)
            
            ne.update_voltage(i,noise[j],dt)
           
    for i,ne in enumerate(neurons):        
        print("Neuron {} spike count {}".format(i,sum(ne.spikes)))
  
    return neurons    

**Q1 :** Run the code in oscillation. Describe the behaviour you observe. How many spikes of neuron A it takes to drive neuron B
to fire? How long is the pause in A's firing after B has inhibited it? Add the plot.

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

Q2 : TODO: Double the strength and duration of the inhibitory influence - in the oscillation
you need spike_strength = [-1000,100] and taus = [30,2]. Run the simulation. How long is the pause in A's firing after B has inhibited it? If you notice any other changes in
behaviour, report them. Add a plot.

In [ ]:
###############################
#### Your code Starts Here ####



#### Your code ends Here ####
###############################

<font color=green>Your Answer</font>

## Exercise 2: Lateral inhibition (1pt)




We will now study a population of 11 neurons arranged on a straight line. All neurons receive
the same injected current. Each neuron inhibits its direct neighbours on the left and on the
right. Therefore the neurons on the left and right edges receive inhibition from only one other
neuron.

In [ ]:
def lateral_inh():
    
    T = 2;  #duration of simulation in sec
    dt=0.001;
    number_of_steps = int(T/dt)
    
    I_const = 50.0 # injected current to each cell
    
    noise = 0 #no noise 
    g_leak= 1
    taus = 20
    spike_strength = -100
    
    n_neurons = 11
    
    #We create the neurons with the class from the previous cell
    neurons = []
    for i in range(n_neurons): 
        neurons.append(Neuron(spike_strength,I_const,g_leak,taus))
        
    #How neurons are connected between them if [0,1] == 1 then conexion from neuron 0 to 1     
    connectivity = np.zeros((n_neurons,n_neurons))
    minus_one = n_neurons-1
    connectivity[0:minus_one,1:n_neurons] += np.eye(minus_one)  
    connectivity[1:n_neurons,0:minus_one] += np.eye(minus_one)  
                  
    for i in range(number_of_steps): 
        
        for j,ne in enumerate(neurons): 
            
            ne.checkspike(i) #Checks if the neuron has generated one spike
        
            neighbours = np.where(connectivity[j,:] == 1)  #Neurons from wich the neuron receives inputs 
            neighbours = np.concatenate(neighbours)
            
            #Calculate incoming spikes 
            in_spikes=[]
            for ngh in neighbours:
                nh_time_spike = np.concatenate(np.nonzero(neurons[ngh].spikes))  #past timepoints with spikes in the neighbour
                in_spikes.append(nh_time_spike)
            in_spikes = np.concatenate(in_spikes)
            
            ne.synaptic_input(in_spikes,i)
            
            ne.update_voltage(i,noise,dt)
    
    spik_count = np.zeros(n_neurons)
    for i,ne in enumerate(neurons):        
        print("Neuron {} spike count {}".format(i,sum(ne.spikes)))
        spik_count[i] = sum(ne.spikes)
        
    
    fig, ax = plt.subplots()
    neuro = range(n_neurons)
    ax.bar(neuro,spik_count, align='center', alpha=0.5, capsize=10)

    ax.set_ylabel('Spikes')
    ax.set_xlabel('Neuron')
    ax.set_xticks(neuro)
    ax.set_title('Lateral inhibition')
    ax.yaxis.grid(True)
  

**Q1 :** Run the code lateral_inh.
It will print out the number of spikes each of the 11 neurons during 2 seconds. Run the code
multiple times. Describe the pattern in the firing rates of neurons. Is this pattern there in every
run? Why do you think this pattern emerges?



In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

Now we will connect the neurons in a circle, not a straight line. This means the leftmost
neuron is now connected to the rightmost and viceversa. This means they will receive inhibition
from two other neurons like everyone else.

In [ ]:
def lateral_circle(display=True):
    
    T = 2;  #duration of simulation in sec
    dt=0.001;
    number_of_steps = int(T/dt)
    
    I_const = 50.0 # injected current to each cell
    
    noise = 0 #no noise 
    g_leak= 1
    taus = 20
    spike_strength = -100
    
    n_neurons = 11
    
    #We create the neurons with the class from the previous cell
    neurons = []
    for i in range(n_neurons): 
        neurons.append(Neuron(spike_strength,I_const,g_leak,taus))
        
    #How neurons are connected between them if [0,1] == 1 then conexion from neuron 0 to 1     
    connectivity = np.zeros((n_neurons,n_neurons))
    minus_one = n_neurons-1
    connectivity[0:minus_one,1:n_neurons] += np.eye(minus_one)  
    connectivity[1:n_neurons,0:minus_one] += np.eye(minus_one)
    connectivity[0,minus_one]=1
    connectivity[minus_one,0]=1
                  
    for i in range(number_of_steps): 
        
        for j,ne in enumerate(neurons): 
            
            ne.checkspike(i) #Checks if the neuron has generated one spike
        
            neighbours = np.where(connectivity[j,:] == 1)  #Neurons from wich the neuron receives inputs 
            neighbours = np.concatenate(neighbours)
            
            #Calculate incoming spikes 
            in_spikes=[]
            for ngh in neighbours:
                nh_time_spike = np.concatenate(np.nonzero(neurons[ngh].spikes))  #past timepoints with spikes in the neighbour
                in_spikes.append(nh_time_spike)
            in_spikes = np.concatenate(in_spikes)
            
            ne.synaptic_input(in_spikes,i)
            
            ne.update_voltage(i,noise,dt)
    
    spik_count = np.zeros(n_neurons)        
    for i,ne in enumerate(neurons): 
        spik_count[i] = sum(ne.spikes)
        if display:
            print("Neuron {} spike count {}".format(i,sum(ne.spikes)))
    
    if not display:  
        return spik_count

def lateral_circle_trials():
    
    trials = 10
    n_neurons = 11
    
    spik_count =np.zeros(n_neurons)
    for i in range(trials):
        spik_count += lateral_circle(False)
   
    spik_count /= trials
    
    fig, ax = plt.subplots()
    neuro = range(n_neurons)
    ax.bar(neuro,spik_count, align='center', alpha=0.5, capsize=10)
    ax.set_xticks(neuro)
    ax.set_ylabel('Spikes')
    ax.set_xlabel('Neuron')
    ax.set_title('Lateral circle')
   
    return spik_count

**Q2 :** Run the code lateral_circle.
It will print out the number of spikes each of the 11 neurons during 2 seconds. Run the code
multiple times. There is a pattern in each individual run, but it's not the same every time. Show some
print outs of spike counts.
To run the same experiment 10 times and average the spike counts, run lateral_circle_trials.
Is there a pattern when we average over many runs? Why does this circular connectivity behave
different from linear?.

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

## Exercise 3: Border effect (1pt)

Our third and final task is to see how the lateral inhibition can enhance our ability to detect
borders. In the lecture you were introduced to the concept and saw some visual illusions caused
by "border enhancement".
To demonstrate this phenomena we will once again use circular connectivity, this time with
15 neurons (more neurons simply to have more "space"). The major difference with previous
exercise is that now neurons at positions 4-12 receive two times stronger injected input than
the rest.
We will first observe the neural behaviour without lateral inhibition and then compare it
with the case where the inhibition is there. The goal is to see that lateral inhibition increases
the difference of activity at the border of the "stimulus" (neuron 3 vs neuron 4 and 12 vs 13).

**Q1 :** First run the function border_effect with spike strength = 0 (synaptic current is
zero).
Show the spike counts of all neurons (and the plot). Report the difference between the spike
count of neurons 3 and 4. Report the difference between spike counts of neurons 12 and 13.

In [ ]:
# Your code
def border_effect(spike_strength = 0.0):
    
    T = 2;  #duration of simulation in sec
    dt=0.001;
    number_of_steps = int(T/dt)
    
    I_const = 25.0 # injected current to each cell
    
    noise = 0 #no noise 
    g_leak= 1
    taus = 20
    
    n_neurons = 15
    
    #We create the neurons with the class from the previous cell
    neurons = []
    # injected current to each cell is doubled for neurons from 4-12
    for i in range(n_neurons): 
        if i>=3 and i<12:
            I_const = 50.0
        else:
            I_const=25.0
        neurons.append(Neuron(spike_strength,I_const,g_leak,taus))
        
    #How neurons are connected between them if [0,1] == 1 then conexion from neuron 0 to 1     
    connectivity = np.zeros((n_neurons,n_neurons))
    minus_one = n_neurons-1
    connectivity[0:minus_one,1:n_neurons] += np.eye(minus_one)  
    connectivity[1:n_neurons,0:minus_one] += np.eye(minus_one)
    connectivity[0,minus_one]=1
    connectivity[minus_one,0]=1
                  
    for i in range(number_of_steps):
        
        for j,ne in enumerate(neurons): 
            
            ne.checkspike(i) #Checks if the neuron has generated one spike
        
            neighbours = np.where(connectivity[j,:] == 1)  #Neurons from wich the neuron receives inputs 
            neighbours = np.concatenate(neighbours)
            
            #Calculate incoming spikes 
            in_spikes=[]
            for ngh in neighbours:
                nh_time_spike = np.concatenate(np.nonzero(neurons[ngh].spikes))  #past timepoints with spikes in the neighbour
                in_spikes.append(nh_time_spike)
            in_spikes = np.concatenate(in_spikes)
            
            ne.synaptic_input(in_spikes,i)
            
            ne.update_voltage(i,noise,dt)
    
    spik_count =np.zeros(n_neurons)   
    for i,ne in enumerate(neurons):        
        print("Neuron {} spike count {}".format(i+1,sum(ne.spikes)))
        spik_count[i] = sum(ne.spikes)

    fig, ax = plt.subplots()
    neuro = range(n_neurons)
    ax.bar(neuro,spik_count, align='center', alpha=0.5, capsize=10)
    ax.set_xticks(neuro)
    labels = range(1,n_neurons+1)
    ax.set_xticklabels(labels)
    ax.set_ylabel('Spikes')
    ax.set_xlabel('Neuron')
    ax.set_title('Border effect')
    plt.plot(spik_count)
    
    return spik_count

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

Now we will turn on the lateral inhibition:

**Q2 :** Run the code border_effect with spike strength = -50.0.
Show the spike counts of all neurons and the plot. Report the difference between the count
of neurons 3 and 4. Report the difference between counts of neurons 12 and 13. Describe
intuitively why is the firing rate of neuron 3 lower than that of neuron 2. Why is firing rate of
neuron 4 higher than that of neuron 5?

In [ ]:
###############################
#### Your code Starts Here ####


##### Your code ends Here #####
###############################

<font color=green>Your Answer</font>

## Exercise 4: The Human Brain Project & The Brain Initiative (1pt)



The two most prominent ongoing brain projects are The Human Brain Project (https://www.humanbrainproject.eu) 
and The Brain Initiative (http://www.nih.gov/science/brain/index.htm). Go through their websites and compose an overview about each of them. It can include
the points like:

+ What is the goal of the project?
+ When is has started?
+ What have they promised to deliver?
+ What have they achieved so far?
+ When is the deadline?
+ What is the value of the results they hope to eventually produce?
+ Is the field of neuroscience done once those projects are complete?
+ Any other information you will find interesting.
In the end you should produce an informative story about the projects, which will allow the
reader to grasp the ideas, the goals and the promises behind those two projects.

<font color=green>Your Answer</font>